Automating Report Preparation for Health Claims Adjudication

Pharmacies send excel file including details of drugs given to patients as free product.
Details client sends is validated against database to confirm eligibility of the patients, then based on drug name and ID, the Client and Group information is pulled.
The report is saved in a master file in Excel which is used by other departments in the company.

In [ ]:
# Read excel file
import pandas as pd

df = pd.read_excel('Client_Data.xlsx', skiprows=1, usecols=[0,1,2,3,4,5,6])

In [ ]:
print(df.head)

In [ ]:
# Get the $ value of amount paid: each drug cost $250

df['Amount Paid'] = (df['Quantity Provided']*250).apply(lambda x: f"${x:,.2f}")

In [ ]:
# Normlize Certificate ID length to 10 digits: Adding leading 0's and removing spaces inbetween Certificate IDs

df['Certificate ID'] = df['Certificate ID'].apply(lambda x:str(x).replace(" ","").zfill(10))

# Concatenate Certificate IDs into a single string for SQL query
certificate_list = ','.join([f"'{certificate}'" for certificate in df['Certificate ID']])

In [ ]:
print(certificate_list)

In [ ]:
# Query the database to check for the validitity and eligibility of the Certificates

import pyodbc

# Establish connection to SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=LAPTOP-906TRAKG\SQLEXPRESS;DATABASE=Eligibility;Trusted_Connection=yes;')


# SQL query to check eligibility
query = f"SELECT Certificate,[Group ID], [Group Name], [Client ID],[Client Name], [Province] FROM [Eligibility].[dbo].[Cert_Eligibility] WHERE [Certificate] IN ({certificate_list})"
eligibility_df = pd.read_sql(query, conn)

conn.close()


In [ ]:
# Merge Client's file with Eligibility data to return valid Certificates

merged_df = pd.merge(df, eligibility_df, left_on='Certificate ID', right_on='Certificate', how='left')

In [ ]:
print(merged_df)

In [ ]:
#Filter out rows where eligibility certificate is missing and drop extra certificate column from eligibility table: invalid certificates not found in the database
valid_certs_df = merged_df[~merged_df['Certificate'].isna()].drop(columns=['Certificate'])

In [ ]:
print(valid_certs_df.head)

In [ ]:
# Add the report date
from datetime import datetime
valid_certs_df['Updated Date'] = datetime.now().strftime('%d-%b-%y')

In [ ]:
# Merge 'First Name' and 'Last Name' into a single 'Patient Name' Column
valid_certs_df['Patient Name'] = valid_certs_df['First Name'] +' ' + valid_certs_df['Last Name'].drop(columns=['First Name','Last Name'])

In [ ]:
# Arrange the columns in order for the report
final_columns = ['Updated Date', 'Certificate ID', 'Patient Name', 'Province', 'Drug Name', 'Drug ID', 
                 'Client ID', 'Client Name', 'Group ID', 'Group Name', 'Date Dispensed', 'Quantity Provided', 'Amount Paid']

In [ ]:
final_df = valid_certs_df[final_columns]

In [ ]:
#Append Data to Excel: 
file_path = 'Final Report.xlsx'

In [ ]:

import os

if os.path.exists(file_path):
    master_df = pd.read_excel(file_path)

In [ ]:
print (master_df)

In [ ]:
#Append Data to Master file
master_df = pd.concat([master_df, final_df], ignore_index=True)


In [ ]:
print(master_df.head)

In [ ]:
# Data exported to Excel

master_df.to_excel(file_path, index=False)